# Feature Selection
## Load the train_reduced and test_reduced data from google drive

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Import package

In [0]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_selection import SelectKBest
import statsmodels.api as sm
import random

## Load train_reduced

The shape of train_reduced should be (68381, 523)

In [0]:
link = 'https://drive.google.com/open?id=18JdiygGsvo9ojuV-qWhn2V40HMMbwcjg'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

18JdiygGsvo9ojuV-qWhn2V40HMMbwcjg


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train_reduced_va.csv')  
df_train = pd.read_csv('train_reduced_va.csv')
print(df_train.shape)


(68381, 523)


## Load test_reduced

The shape of train_reduced should be (29307, 523)

In [0]:
link = 'https://drive.google.com/open?id=1FILBSURi1WGVoO-8VNu6i2D0LV3NAmqz'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1FILBSURi1WGVoO-8VNu6i2D0LV3NAmqz


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('test_reduced_va.csv')  
df_test = pd.read_csv('test_reduced_va.csv')
print(df_test.shape)

(29307, 523)


## Use train dataset to select the features
Rename the training data to df3 for avoiding reload the original dataset from the beginning. 

In [0]:
df3 = df_train

Remove `ee` from the train data and create a new dataframe called df4

In [0]:
df4 = df3.loc[:,df3.columns != 'ee']
df4.head()

,ncat,oldmodel,cr_vio_s_1y,cr_murd_s_1y,cr_rape_s_1y,cr_robb_s_1y,cr_agg_s_1y,cr_ppty_s_1y,cr_bur_s_1y,cr_lacy_s_1y,cr_mtv_s_1y,cr_arson_s_1y,cr_vio_r_st_1y,cr_murd_r_st_1y,cr_rape_r_st_1y,cr_robb_r_st_1y,cr_agg_r_st_1y,cr_ppty_r_st_1y,cr_bur_r_st_1y,cr_lacy_r_st_1y,cr_mtv_r_st_1y,cr_arson_r_st_1y,cr_vio_r_nt_1y,cr_murd_r_nt_1y,cr_rape_r_nt_1,cr_robb_r_nt_1y,cr_agg_r_nt_1y,cr_ppty_r_nt_1y,cr_bur_r_nt_1y,cr_lacy_r_nt_1y,cr_mtv_r_nt_1y,cr_arson_r_nt_1y,cr_total_1y,cr_vio_r_1y,cr_ppty_r_1y,cr_murd_r_1y,cr_rape_r_1y,cr_robb_r_1y,cr_agg_r_1y,cr_bur_r_1y,...,ltg_sm_strikes_pop_20y,ltg_sm_events_pop_20y,ltg_sm_strikes_ppt_20y,ltg_sm_events_ppt_20y,ltg_fa_strikes_20y,ltg_fa_events_20y,ltg_fa_strike_event_20y,ltg_fa_strikes_m_sq_20y,ltg_fa_events_m_sq_20y,ltg_fa_strikes_pop_20y,ltg_fa_events_pop_20y,ltg_fa_strikes_ppt_20y,ltg_fa_events_ppt_20y,ltg_wt_strikes_20y,ltg_wt_events_20y,ltg_wt_strike_event_20y,ltg_wt_strikes_m_sq_20y,ltg_wt_events_m_sq_20y,ltg_wt_strikes_pop_20y,ltg_wt_events_pop_20y,ltg_wt_strikes_ppt_20y,ltg_wt_events_ppt_20y,ltg_st_strikes_20y,ltg_st_events_20y,ltg_st_strike_event_20y,ltg_st_strikes_m_sq_20y,ltg_st_events_m_sq_20y,ltg_st_strikes_pop_20y,ltg_st_events_pop_20y,ltg_st_strikes_ppt_20y,ltg_st_events_ppt_2oy,ltg_nt_strikes_20y,ltg_nt_events_20y,ltg_nt_strike_event_20y,ltg_nt_strikes_m_sq_20y,ltg_nt_events_m_sq_20y,ltg_nt_strikes_pop_20y,ltg_nt_events_pop_20y,ltg_nt_strikes_ppt_20y,ltg_nt_events_ppt_20y
0,0,339.94,34.000000,0.000000,6.000000,1.000000,27.000000,205.000000,35.000000,159.000000,11.000000,0.000000,0.001094,0.000000,0.001941,0.000183,0.001210,0.001795,0.001891,0.001831,0.001240,0.000000,0.000282,0.000000,0.000564,0.000029,0.000367,0.000366,0.000417,0.000377,0.000203,0.000000,239.000000,0.142259,0.857741,0.000000,0.176471,0.029412,0.794118,0.170732,...,2.695825,0.299776,0.010720,0.001192,113,21,5.380952,0.461113,0.085694,0.406171,0.075483,0.001615,0.000300,23,4,5.75,0.281565,0.048968,0.248016,0.043133,0.000986,0.000172,0.002061,0.002081,0.990319,3.105783e+06,3.136144e+06,279.928134,282.664560,0.504446,0.509377,0.000002,0.000004,0.552970,8.207351e+06,1.484232e+07,56.245482,101.715327,0.080725,0.145985
1,0,248.77,13.000000,0.000000,0.000000,1.000000,12.000000,88.000000,11.000000,69.000000,8.000000,1.000000,0.000418,0.000000,0.000000,0.000183,0.000538,0.000770,0.000594,0.000795,0.000902,0.002730,0.000108,0.000000,0.000000,0.000029,0.000163,0.000157,0.000131,0.000164,0.000148,0.000339,101.000000,0.128713,0.871287,0.000000,0.000000,0.076923,0.923077,0.125000,...,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
2,0,523.65,62.406412,0.382167,6.210917,11.006402,44.806926,229.430259,37.186238,174.420897,17.823124,0.735766,0.002009,0.002009,0.002009,0.002009,0.002009,0.002009,0.002009,0.002009,0.002009,0.002009,0.000518,0.000206,0.000584,0.000320,0.000609,0.000410,0.000443,0.000413,0.000330,0.000250,291.836671,0.213840,0.786160,0.006124,0.099524,0.176367,0.717986,0.162081,...,5.609614,0.582169,0.018757,0.001947,158,23,6.869565,0.719694,0.104766,0.876026,0.127523,0.002929,0.000426,8,5,1.60,0.109321,0.068325,0.133067,0.083167,0.000445,0.000278,0.002240,0.002156,1.039051,4.367461e+06,4.203316e+06,543.966396,523.522084,0.824232,0.793254,0.000003,0.000005,0.580180,1.154146e+07,1.989289e+07,109.298240,188.386616,0.131900,0.227342
3,0,213.16,1271.000000,4.000000,49.000000,290.000000,928.000000,3979.000000,849.000000,2790.000000,340.000000,9.000000,0.040911,0.021025,0.015848,0.052927,0.041603,0.034837,0.045862,0.032131,0.038319,0.024571,0.010547,0.002159,0.004607,0.008423,0.012611,0.007106,0.010114,0.006612,0.006288,0.003055,5250.000000,0.242095,0.757905,0.003147,0.038552,0.228167,0.730134,0.213370,...,4.397966,0.421974,0.014461,0

## Identify x and y and fit the OLS model
Keep `oldmodel` and other crime and lightning variables in x, `ncat` as y


In [0]:
x = df4.loc[:,df4.columns != 'ncat']
y = df4.loc[:,'ncat']
#Adding constant column of ones, mandatory for sm.OLS model
X_1 = sm.add_constant(x)
#Fitting sm.OLS model
model = sm.OLS(y,X_1).fit()
model.pvalues

const                      4.144667e-01
oldmodel                  2.172131e-154
cr_vio_s_1y                3.941105e-01
cr_murd_s_1y               7.421110e-01
cr_rape_s_1y               4.180180e-01
                              ...      
ltg_nt_events_m_sq_20y     3.859618e-01
ltg_nt_strikes_pop_20y     2.774487e-01
ltg_nt_events_pop_20y      2.986849e-01
ltg_nt_strikes_ppt_20y     1.566920e-01
ltg_nt_events_ppt_20y      1.757171e-01
Length: 522, dtype: float64

## Loop over the p-value from the model and eliminate those insignificant variales(p-value > 0.05)

In [0]:
#Backward Elimination
random.seed(45)
cols = list(x.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = x[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

['oldmodel', 'cr_robb_s_1y', 'cr_agg_r_1y', 'cr_mtv_r_st_3y', 'cr_bur_r_3y', 'cr_lacy_r_3y', 'cr_mtv_r_3y', 'cr_robb_s_5y', 'cr_ppty_s_5y', 'cr_bur_s_5y', 'cr_agg_r_5y', 'cr_lacy_s_10y', 'cr_ppty_r_st_10y', 'cr_bur_r_st_10y', 'cr_mtv_r_st_10y', 'cr_ppty_r_nt_10y', 'cr_vio_r_10y', 'cr_ppty_r_10y', 'cr_murd_r_10y', 'cr_lacy_r_10y', 'cr_bur_r_st_20y', 'cr_lacy_r_st_20y', 'cr_lacy_r_20y', 'cr_mtv_r_20y', 'ltg_sm_events_3y', 'ltg_fa_events_3y', 'ltg_strikes_5y', 'ltg_wt_strikes_5y', 'ltg_wt_events_5y', 'ltg_strikes_10y', 'ltg_events_10y', 'ltg_fa_strikes_10y', 'ltg_wt_strikes_10y', 'ltg_nt_events_10y']


In [0]:
va_withold= selected_features_BE
len(va_withold)

34

### Extract from Train

In [0]:
# create indices
indices_withold = list(va_withold)
selected_withold = df_train.loc[:, indices_withold]


In [0]:
len(selected_withold.columns)

34

Selected total 34 features from the model

### Feature treatment (using winsorize to cut in 99th persentile)

In [0]:
## winsorize result_cr
from scipy import stats
for col in selected_withold.columns:
  selected_withold[col] = stats.mstats.winsorize(selected_withold[col], limits=[0.01,0.01], inplace=True) 

selected_withold.describe()

,oldmodel,cr_robb_s_1y,cr_agg_r_1y,cr_mtv_r_st_3y,cr_bur_r_3y,cr_lacy_r_3y,cr_mtv_r_3y,cr_robb_s_5y,cr_ppty_s_5y,cr_bur_s_5y,cr_agg_r_5y,cr_lacy_s_10y,cr_ppty_r_st_10y,cr_bur_r_st_10y,cr_mtv_r_st_10y,cr_ppty_r_nt_10y,cr_vio_r_10y,cr_ppty_r_10y,cr_murd_r_10y,cr_lacy_r_10y,cr_bur_r_st_20y,cr_lacy_r_st_20y,cr_lacy_r_20y,cr_mtv_r_20y,ltg_sm_events_3y,ltg_fa_events_3y,ltg_strikes_5y,ltg_wt_strikes_5y,ltg_wt_events_5y,ltg_strikes_10y,ltg_events_10y,ltg_fa_strikes_10y,ltg_wt_strikes_10y,ltg_nt_events_10y
count,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000,68381.000000
mean,271.770628,32.972866,0.626502,0.006157,0.167832,0.753839,0.077381,201.914129,3170.222431,614.253554,0.622717,5047.660560,0.005395,0.005468,0.006037,0.001015,0.129474,0.870526,0.009179,0.722658,0.005271,0.005040,0.689868,0.107822,8.182916,1.635293,252.533759,1.901961,0.597257,476.381919,43.369298,38.859420,2.288647,0.000001
std,87.075278,80.331163,0.206957,0.013112,0.059769,0.077215,0.037390,460.340360,4918.143172,1163.104219,0.144836,7246.121759,0.009228,0.009758,0.013042,0.001585,0.058757,0.058757,0.009709,0.074385,0.009090,0.007796,0.072595,0.045040,12.919301,2.664203,482.468949,4.622188,1.190022,878.481987,69.763069,69.634532,5.307470,0.000002
min,143.380000,0.000000,0.000000,0.000018,0.044872,0.500000,0.006536,0.000000,74.000000,11.655256,0.166667,135.228818,0.000129,0.000116,0.000035,0.000027,0.020833,0.720715,0.000000,0.498173,0.000227,0.000237,0.509938,0.023585,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,209.440000,1.000000,0.549121,0.000663,0.130067,0.717447,0.048433,10.000000,585.000000,99.000000,0.499857,1072.000000,0.001039,0.000994,0.000660,0.000204,0.083151,0.839671,0.003032,0.676624,0.001030,0.001095,0.632794,0.068651,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,253.140000,7.798894,0.629310,0.001833,0.174871,0.746695,0.076412,48.000000,1493.000000,247.209514,0.636364,2523.843168,0.002197,0.002107,0.001754,0.000485,0.132422,0.867578,0.006204,0.716352,0.002159,0.002274,0.675447,0.108507,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,313.960000,26.106575,0.758333,0.004285,0.179828,0.807595,0.106630,168.058996,3382.000000,570.879994,0.737121,5913.000000,0.004970,0.004618,0.004320,0.001106,0.160329,0.916849,0.015438,0.777995,0.004545,0.005067,0.743936,0.157055,17.000000,3.000000,339.000000,1.000000,1.000000,721.000000,93.000000,75.000000,1.000000,0.000002
max,591.310000,475.000000,1.000000,0.080056,0.371368,0.928205,0.204114,2708.000000,24827.000000,6675.000000,0.900000,36177.000000,0.056491,0.050710,0.084846,0.008443,0.279285,0.979167,0.066696,0.888350,0.050848,0.046513,0.857255,0.209754,52.000000,11.000000,2392.000000,25.000000,6.000000,4586.000000,330.000000,384.000000,28.000000,0.000008


### Add `ncat` and `ee` to the new dataframe called selected_34, and write to csv file and rename the file name 'selected_34_tr.csv'.

In [0]:
## add ncat and ee and oldmodel in the result_lgt
selected_34 = selected_withold
selected_34['ncat'] = df4['ncat']
selected_34['oldmodel']=df4['oldmodel']
selected_34['ee']=df3['ee']
print(selected_34)

       oldmodel  cr_robb_s_1y  cr_agg_r_1y  ...  ltg_nt_events_10y  ncat      ee
0        339.94      1.000000     0.794118  ...           0.000002     0  1.0000
1        248.77      1.000000     0.923077  ...           0.000000     0  0.0438
2        523.65     11.006402     0.717986  ...           0.000002     0  0.7973
3        213.16    290.000000     0.730134  ...           0.000002     0  1.0000
4        214.90      1.000000     0.642857  ...           0.000000     0  1.0000
...         ...           ...          ...  ...                ...   ...     ...
68376    153.19    475.000000     0.534813  ...           0.000000   805  0.2137
68377    246.23     96.714373     0.584129  ...           0.000000   330  1.0000
68378    273.58    475.000000     0.534813  ...           0.000000  1373  0.9123
68379    195.45     91.000000     0.528428  ...           0.000000   275  0.8493
68380    235.98    475.000000     0.534813  ...           0.000000  1500  1.0000

[68381 rows x 36 columns]


In [0]:
## write the result_cr1 to csv
selected_34.to_csv('selected_34_tr.csv',index = False)

### Extract 34 selected features from test_reduced data and create a new dataframe with 34 features called df7



In [0]:
# create indices
indices_withold = list(va_withold)
selected_withold = df_test.loc[:, indices_withold]

### Feature treatment (using winsorize to cut in 99th persentile)

In [0]:
## winsorize result_cr
from scipy import stats
for col in selected_withold.columns:
  selected_withold[col] = stats.mstats.winsorize(selected_withold[col], limits=[0.01,0.01], inplace=True) 

selected_withold.describe()

,oldmodel,cr_robb_s_1y,cr_agg_r_1y,cr_mtv_r_st_3y,cr_bur_r_3y,cr_lacy_r_3y,cr_mtv_r_3y,cr_robb_s_5y,cr_ppty_s_5y,cr_bur_s_5y,cr_agg_r_5y,cr_lacy_s_10y,cr_ppty_r_st_10y,cr_bur_r_st_10y,cr_mtv_r_st_10y,cr_ppty_r_nt_10y,cr_vio_r_10y,cr_ppty_r_10y,cr_murd_r_10y,cr_lacy_r_10y,cr_bur_r_st_20y,cr_lacy_r_st_20y,cr_lacy_r_20y,cr_mtv_r_20y,ltg_sm_events_3y,ltg_fa_events_3y,ltg_strikes_5y,ltg_wt_strikes_5y,ltg_wt_events_5y,ltg_strikes_10y,ltg_events_10y,ltg_fa_strikes_10y,ltg_wt_strikes_10y,ltg_nt_events_10y
count,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,29307.000000,2.930700e+04
mean,271.490837,32.676025,0.626419,0.006160,0.167968,0.753951,0.077253,200.711290,3170.699728,612.747647,0.623659,5055.909630,0.005424,0.005492,0.006035,0.001016,0.129521,0.870479,0.009321,0.722826,0.005276,0.005055,0.689702,0.107641,8.108438,1.633193,252.481523,1.882417,0.594500,475.672877,43.077081,38.569966,2.270038,9.940629e-07
std,86.497666,79.683978,0.208392,0.013110,0.060132,0.076943,0.037338,459.241852,4930.300980,1159.402086,0.143979,7267.016244,0.009304,0.009787,0.013038,0.001586,0.058791,0.058791,0.010514,0.074009,0.009038,0.007818,0.071972,0.044694,12.872686,2.733124,484.551574,4.614738,1.187572,882.989377,69.882801,69.707984,5.298247,1.612642e-06
min,144.200000,0.000000,0.000000,0.000018,0.044872,0.500000,0.006536,0.000000,74.000000,12.000000,0.187500,131.590417,0.000133,0.000120,0.000035,0.000026,0.019776,0.720715,0.000000,0.498173,0.000234,0.000245,0.509938,0.023886,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,209.460000,1.000000,0.549121,0.000661,0.130067,0.717447,0.048387,10.000000,585.000000,99.000000,0.499857,1072.000000,0.001039,0.000990,0.000655,0.000203,0.083151,0.839671,0.003032,0.676624,0.001037,0.001091,0.632794,0.068651,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
50%,253.310000,7.767238,0.633333,0.001833,0.174871,0.746695,0.076282,48.000000,1482.319819,246.519078,0.642857,2507.000000,0.002166,0.002099,0.001754,0.000481,0.132422,0.867578,0.006110,0.716352,0.002153,0.002250,0.676824,0.108348,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
75%,313.605000,26.000000,0.760000,0.004285,0.179878,0.807317,0.106630,168.000000,3335.000000,570.879994,0.736208,5898.000000,0.004970,0.004618,0.004285,0.001104,0.160329,0.916849,0.015438,0.777995,0.004545,0.005067,0.743603,0.157055,17.000000,3.000000,337.000000,1.000000,1.000000,715.000000,92.000000,74.000000,1.000000,2.123027e-06
max,591.960000,475.000000,1.000000,0.080056,0.377551,0.928205,0.204114,2708.000000,24827.000000,6675.000000,0.896552,36177.000000,0.056491,0.050710,0.084846,0.008443,0.279285,0.980224,0.076923,0.886131,0.050848,0.046513,0.852946,0.209754,51.000000,13.000000,2392.000000,25.000000,6.000000,4631.000000,334.000000,385.000000,28.000000,7.707509e-06


### Add `ncat` and `ee` to the new dataframe called selected_34, and write to csv file and rename the file name 'selected_34_te.csv'.

In [0]:
## add ncat and ee and oldmodel in the result_lgt
selected_34 = selected_withold
selected_34['ncat'] = df_test['ncat']
selected_34['oldmodel']=df_test['oldmodel']
selected_34['ee']=df_test['ee']
print(selected_34)

       oldmodel  cr_robb_s_1y  cr_agg_r_1y  ...  ltg_nt_events_10y   ncat      ee
0        252.23     22.000000     0.674699  ...           0.000002      0  0.3507
1        318.18    102.000000     0.619469  ...           0.000000      0  0.0027
2        414.08      0.000000     0.769231  ...           0.000002      0  0.1776
3        590.09     11.500701     0.717986  ...           0.000000      0  1.0000
4        214.92     11.000000     0.823899  ...           0.000000      0  1.0000
...         ...           ...          ...  ...                ...    ...     ...
29302    266.84    221.000000     0.528180  ...           0.000000  17705  1.0000
29303    228.90    475.000000     0.534813  ...           0.000000    375  1.0000
29304    167.57     29.960838     0.584129  ...           0.000000  36354  0.7753
29305    383.37     42.336611     0.584129  ...           0.000000  29653  1.0000
29306    269.83     29.241053     0.584129  ...           0.000000   5515  1.0000

[29307 rows x 3

In [0]:
## write the result_cr1 to csv
selected_34.to_csv('selected_34_te.csv',index = False)